# Write Data to Database

This notebook was used to generate the code needed in the lambda function to write data from a csv file deposited in an s3 bucket to the database.

In [1]:
import pandas as pd
import numpy as np

## Connect to Database

In [2]:
import mysql.connector

In [3]:
endpoint = "seratestdatabase.c4cjk1vto1om.us-east-2.rds.amazonaws.com"
port = "3306"
usr = "admin"
pswd = "seracapstone"
region = "us-east-2b"
dbname = "teachsim"

In [4]:
cnx = mysql.connector.connect(user=usr, password=pswd, host=endpoint, database=dbname)

In [5]:
cursor = cnx.cursor(buffered=True)

## Test Connection

In [8]:
cursor.execute("SELECT id_participant, id_section, id_site, id_year, fb_treat_cond, s18_treat_cond FROM Identifiers WHERE id_participant LIKE 'testID%'")
#cursor.execute("SELECT * FROM Identifiers;")
result = cursor.fetchall()
colnames = [x[0] for x in cursor.description]
pd.DataFrame(result, columns=colnames)

,id_participant,id_section,id_site,id_year,fb_treat_cond,s18_treat_cond
0,testID4,2.0,1,1718.0,1.0,4.0
1,testID5,1.0,1,2021.0,0.0,5.0
2,testID6,1.0,1,1718.0,1.0,4.0
3,testID7,2.0,1,2021.0,0.0,5.0


In [16]:
cursor.execute("REPLACE INTO Identifiers (id_participant) VALUES ('testID3');")

In [7]:
cursor.execute("DELETE FROM Identifiers;")

## Set Working Directory

In [11]:
import os
os.getcwd()

'/Users/jmachita03/Documents/GitHub/MSDS_SERA_capstone/RDS'

In [12]:
#os.chdir("/Users/jmachita03/Desktop/Capstone/RDS Set Up")

In [13]:
os.chdir('/Users/jmachita03/Documents/GitHub/MSDS_SERA_capstone/RDS')

## Functions

In [14]:
def getColumnNames(tableName, cursor):
    query = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'" + tableName + "'"
    cursor.execute(query)
    result = cursor.fetchall()
    result2 = [x[0] for x in result]
    
    return result2

In [15]:
def overlapColumns(cols1, cols2):
    overlap = []
    for col in cols1:
        if col in cols2:
            overlap = overlap + [col]
    return overlap

In [16]:
def getExistingIds(tableName, cursor):
    query = "SELECT id_participant FROM " + str(tableName)
    cursor.execute(query)
    result = cursor.fetchall()
    existing_ids = [x[0] for x in result]
    return existing_ids

In [17]:
# encompasses a combo of the replaceQueries and updateQueries functions
def list_queries(df, columns, tableName, existing_ids):
    queries = []
    for index, row in df.iterrows():
        # if the row for that participant already exists, just update it
        if row['id_participant'] in existing_ids:
            req_update = False
            query = "UPDATE " + tableName + " SET "
            for col in columns:
                newval = row[col]
                if type(newval) == str:
                    query = query + col + " = '" + newval + "' , "
                    req_update = True
                elif np.isnan(float(newval)) == False: 
                    query = query + col + " = " + str(newval) + " , "
                    req_update = True
            if req_update == True:
                query = query[:-2]
                query = query + "WHERE id_participant = '" + row["id_participant"] + "';"
                queries = queries + [query]
        
        # if that participant is not in the database, create/replace the row
        else:
            req_update = False
            query = "REPLACE INTO " +  tableName + " "
            cols = "("
            vals = "("
            for col in columns:
                val = row[col] 
                if type(val) == str:
                    cols = cols + str(col) + ", "
                    vals = vals + "'" + val + "', "
                    req_update = True
                elif np.isnan(float(val)) == False: 
                    cols = cols + str(col) + ", "
                    vals = vals + str(val) + ", "
                    req_update = True
            query = query + cols[:-2] + ") " + 'VALUES ' + vals[:-2] + ");"
            if req_update == True:
                queries = queries + [query]
                existing_ids = existing_ids + [row['id_participant']]
            
    return queries

In [18]:
def generateQueries(df, tableName, cursor):
    df_cols = df.columns
    tab_cols = getColumnNames(tableName, cursor)
    overlap = overlapColumns(df_cols, tab_cols)
    
    existing_ids = getExistingIds(tableName, cursor)
    
    query_list = list_queries(df, overlap, tableName, existing_ids)
    
    return query_list

In [19]:
pd.__file__

'/Users/jmachita03/opt/anaconda3/lib/python3.7/site-packages/pandas/__init__.py'

## Get Id_Participants Already in Table

In [21]:
cursor.execute("SELECT id_participant FROM Identifiers")
result = cursor.fetchall()
existing_ids = [x[0] for x in result]
existing_ids

['testID3']

## Import Cleaned Dataset

In [22]:
data = pd.read_csv("outcome_merged.csv")

In [23]:
cleaned_cols = list(data.columns)

In [24]:
uploaded_data = data.dropna(axis='rows', subset=['id_participant'])

In [26]:
uploaded_data.id_participant[0]

'1_1819_2_67'

## Test generateQueries function

In [27]:
quer = generateQueries(uploaded_data, "Identifiers", cursor)

In [28]:
quer

["REPLACE INTO Identifiers (fb_treat_cond, id_participant, id_section, id_site, id_year) VALUES (0.0, '1_1819_2_67', 2.0, 1.0, 1819.0);",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_1819_2_67' , id_section = 2.0 , id_site = 1.0 , id_year = 1819.0 WHERE id_participant = '1_1819_2_67';",
 "REPLACE INTO Identifiers (fb_treat_cond, id_participant, id_section, id_site, id_year) VALUES (0.0, '1_1819_3_86', 3.0, 1.0, 1819.0);",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_1819_3_86' , id_section = 3.0 , id_site = 1.0 , id_year = 1819.0 WHERE id_participant = '1_1819_3_86';",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_1819_3_86' , id_section = 3.0 , id_site = 1.0 , id_year = 1819.0 WHERE id_participant = '1_1819_3_86';",
 "REPLACE INTO Identifiers (fb_treat_cond, id_participant, id_section, id_site, id_year) VALUES (0.0, '1_1819_4_108', 4.0, 1.0, 1819.0);",
 "UPDATE Identifiers SET fb_treat_cond = 0.0 , id_participant = '1_18

## Test Loop of Queries

In [29]:
tables = ['Identifiers', 'Survey_Measures', 'Participant_Measures', 'Performance_Measures']
for table in tables:
    queries = generateQueries(uploaded_data, table, cursor)
    for query in queries:
        cursor.execute(query)

## Playing Around with Schema

In [33]:
cursor.execute("SELECT column_name, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'Identifiers'")

In [34]:
cursor.fetchall()

[('br_treat_cond', b'float'),
 ('br_treat_cond_mr', b'float'),
 ('br_treat_cond_iat', b'float'),
 ('fb_treat_cond', b'float'),
 ('f17_treat_cond', b'float'),
 ('s18_treat_cond', b'float'),
 ('fb_treat_cond_iat', b'float'),
 ('id_participant', b'varchar'),
 ('id_section', b'float'),
 ('id_site', b'varchar'),
 ('id_study', b'float'),
 ('id_year', b'float')]

## Close Database Connection

In [7]:
cnx.commit()
cnx.close()